In [1]:
!pip install requests psycopg2-binary beautifulsoup4 selenium webdriver-manager pandas Pillow


In [2]:
# ═══════════════════════════════════════════════════════════
# CELLULE 2 — IMPORTS & CONFIGURATION
# ═══════════════════════════════════════════════════════════

import os, re, time, json, random, requests, psycopg2
from bs4 import BeautifulSoup
from collections import Counter
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager

DB_CONFIG = {
    "host":     "localhost",
    "database": "estate_mind_db",
    "user":     "postgres",
    "password": "admin"
}

MEDIA_ROOT        = r"C:\Users\chtou\Desktop\scrapping\media_fidari"
TARGET_PER_TYPE   = 200
MAX_PAGES_INITIAL = 20
MAX_PAGES_DAILY   = 3
BASE_URL          = "https://fi-dari.tn"
B                 = "[[37.649,7.778],[30.107,11.953]]"

CATEGORIES = {
    f"search?objectif=vendre&categorie=Appartement&usage=Habitation&bounds={B}":           ("sale", "apartment"),
    f"search?objectif=louer&categorie=Appartement&usage=Tout+type+de+location&bounds={B}": ("rent", "apartment"),
    f"search?objectif=vendre&categorie=Maison&usage=Habitation&bounds={B}":                ("sale", "house"),
    f"search?objectif=louer&categorie=Maison&usage=Tout+type+de+location&bounds={B}":      ("rent", "house"),
    f"search?objectif=vendre&categorie=Terrain&bounds={B}":                                ("sale", "land"),
    f"search?objectif=louer&categorie=Terrain&bounds={B}":                                 ("rent", "land"),
    f"search?objectif=vendre&categorie=Bureau&usage=Professionnels&bounds={B}":            ("sale", "office"),
    f"search?objectif=louer&categorie=Bureau&usage=Bureaux+et+commerce&bounds={B}":        ("rent", "office"),
}

PROPERTY_CONFIG = {
    "apartment": {"has_rooms": True,  "has_surface": True},
    "house":     {"has_rooms": True,  "has_surface": True},
    "land":      {"has_rooms": False, "has_surface": True},
    "office":    {"has_rooms": False, "has_surface": True},
}

# ── Carte géographique complète Tunisie ─────────────────────────────────────
# Chaque ville/délégation est mappée vers (zone, gouvernorat_officiel)
CITY_TO_GEO = {
    # Grand Tunis
    "tunis":            ("grand_tunis", "Tunis"),
    "ariana":           ("grand_tunis", "Ariana"),
    "ben arous":        ("grand_tunis", "Ben Arous"),
    "manouba":          ("grand_tunis", "Manouba"),
    "la marsa":         ("grand_tunis", "Tunis"),
    "marsa":            ("grand_tunis", "Tunis"),
    "carthage":         ("grand_tunis", "Tunis"),
    "la goulette":      ("grand_tunis", "Tunis"),
    "bardo":            ("grand_tunis", "Tunis"),
    "ezzahra":          ("grand_tunis", "Ben Arous"),
    "hammam lif":       ("grand_tunis", "Ben Arous"),
    "hammam chatt":     ("grand_tunis", "Ben Arous"),
    "bou mhel":         ("grand_tunis", "Ben Arous"),
    "el aouina":        ("grand_tunis", "Tunis"),
    "el menzah":        ("grand_tunis", "Ariana"),
    "soukra":           ("grand_tunis", "Ariana"),
    "raoued":           ("grand_tunis", "Ariana"),
    "ennasr":           ("grand_tunis", "Ariana"),
    "mrezga":           ("grand_tunis", "Ariana"),
    "centre urbain":    ("grand_tunis", "Tunis"),
    "lac 1":            ("grand_tunis", "Tunis"),
    "lac 2":            ("grand_tunis", "Tunis"),
    "les berges du lac":("grand_tunis", "Tunis"),
    "ain zaghouan":     ("grand_tunis", "Tunis"),
    "el ghazela":       ("grand_tunis", "Ariana"),
    "borj louzir":      ("grand_tunis", "Ariana"),
    "mnihla":           ("grand_tunis", "Ariana"),
    "kalaat landlous":  ("grand_tunis", "Manouba"),
    "douar hicher":     ("grand_tunis", "Manouba"),
    "den den":          ("grand_tunis", "Manouba"),
    "oued ellil":       ("grand_tunis", "Manouba"),
    "jedaida":          ("grand_tunis", "Manouba"),
    "tebourba":         ("grand_tunis", "Manouba"),
    "mourouj":          ("grand_tunis", "Ben Arous"),
    "medina jadida":    ("grand_tunis", "Ben Arous"),
    "megrine":          ("grand_tunis", "Ben Arous"),
    "hammam el ghezaz": ("grand_tunis", "Ben Arous"),
    "fouchana":         ("grand_tunis", "Ben Arous"),
    "mornag":           ("grand_tunis", "Ben Arous"),
    # Nord
    "bizerte":          ("nord", "Bizerte"),
    "nabeul":           ("nord", "Nabeul"),
    "hammamet":         ("nord", "Nabeul"),
    "beja":             ("nord", "Béja"),
    "jendouba":         ("nord", "Jendouba"),
    "zaghouan":         ("nord", "Zaghouan"),
    "kelibia":          ("nord", "Nabeul"),
    "korba":            ("nord", "Nabeul"),
    "menzel bourguiba": ("nord", "Bizerte"),
    "grombalia":        ("nord", "Nabeul"),
    "ain draham":       ("nord", "Jendouba"),
    "tabarka":          ("nord", "Jendouba"),
    "menzel temime":    ("nord", "Nabeul"),
    "el haouaria":      ("nord", "Nabeul"),
    "dar chaabane":     ("nord", "Nabeul"),
    "beni khiar":       ("nord", "Nabeul"),
    "kélibia":          ("nord", "Nabeul"),
    "soliman":          ("nord", "Nabeul"),
    "bou argoub":       ("nord", "Nabeul"),
    "menzel bouzaiane": ("nord", "Nabeul"),
    "takelsa":          ("nord", "Nabeul"),
    # Sahel
    "sousse":           ("sahel", "Sousse"),
    "monastir":         ("sahel", "Monastir"),
    "mahdia":           ("sahel", "Mahdia"),
    "msaken":           ("sahel", "Sousse"),
    "el jem":           ("sahel", "Mahdia"),
    "hergla":           ("sahel", "Sousse"),
    "akouda":           ("sahel", "Sousse"),
    "kantaoui":         ("sahel", "Sousse"),
    "port el kantaoui": ("sahel", "Sousse"),
    "kalaa kebira":     ("sahel", "Sousse"),
    "kalaa sghira":     ("sahel", "Sousse"),
    "enfidha":          ("sahel", "Sousse"),
    "hamammet":         ("sahel", "Nabeul"),
    "ksar hellal":      ("sahel", "Monastir"),
    "moknine":          ("sahel", "Monastir"),
    "bekalta":          ("sahel", "Monastir"),
    "jemmal":           ("sahel", "Monastir"),
    "bembla":           ("sahel", "Monastir"),
    "bou merdes":       ("sahel", "Monastir"),
    "chebba":           ("sahel", "Mahdia"),
    "ksour essef":      ("sahel", "Mahdia"),
    # Centre
    "kairouan":         ("centre", "Kairouan"),
    "kasserine":        ("centre", "Kasserine"),
    "sidi bouzid":      ("centre", "Sidi Bouzid"),
    "siliana":          ("centre", "Siliana"),
    "gafsa":            ("centre", "Gafsa"),
    "sbeitla":          ("centre", "Kasserine"),
    "sbiba":            ("centre", "Kasserine"),
    "thala":            ("centre", "Kasserine"),
    # Sud
    "sfax":             ("sud", "Sfax"),
    "gabes":            ("sud", "Gabès"),
    "gabès":            ("sud", "Gabès"),
    "jerba":            ("sud", "Médenine"),
    "djerba":           ("sud", "Médenine"),
    "medenine":         ("sud", "Médenine"),
    "médenine":         ("sud", "Médenine"),
    "tataouine":        ("sud", "Tataouine"),
    "tozeur":           ("sud", "Tozeur"),
    "kebili":           ("sud", "Kébili"),
    "kébili":           ("sud", "Kébili"),
    "zarzis":           ("sud", "Médenine"),
    "ben gardane":      ("sud", "Médenine"),
    "el hamma":         ("sud", "Gabès"),
    "matmata":          ("sud", "Gabès"),
    "ghomrassen":       ("sud", "Tataouine"),
    "remada":           ("sud", "Tataouine"),
    "ghoumrassen":      ("sud", "Tataouine"),
    "nefta":            ("sud", "Tozeur"),
    "douz":             ("sud", "Kébili"),
    "sfax sud":         ("sud", "Sfax"),
    "sfax ville":       ("sud", "Sfax"),
    "thyna":            ("sud", "Sfax"),
    "sakiet eddaier":   ("sud", "Sfax"),
    "sakiet ezzit":     ("sud", "Sfax"),
    "el ain":           ("sud", "Sfax"),
    "agareb":           ("sud", "Sfax"),
}

# Table gouvernorat → zone (fallback)
GOUVERNORAT_ZONE = {
    "Tunis":      "grand_tunis", "Ariana":    "grand_tunis",
    "Ben Arous":  "grand_tunis", "Manouba":   "grand_tunis",
    "Bizerte":    "nord",        "Nabeul":    "nord",
    "Béja":       "nord",        "Jendouba":  "nord",
    "Zaghouan":   "nord",
    "Sousse":     "sahel",       "Monastir":  "sahel",
    "Mahdia":     "sahel",
    "Kairouan":   "centre",      "Kasserine": "centre",
    "Sidi Bouzid":"centre",      "Siliana":   "centre",
    "Gafsa":      "centre",
    "Sfax":       "sud",         "Gabès":     "sud",
    "Médenine":   "sud",         "Tataouine": "sud",
    "Tozeur":     "sud",         "Kébili":    "sud",
}

FEATURE_MAPPING = {
    "Climatisation":"has_air_conditioning","Climatiseur":"has_air_conditioning",
    "Chauffage central":"has_central_heating","Chauffage":"has_heating",
    "Ascenseur":"has_elevator","Jardin":"has_garden","Piscine":"has_pool",
    "Terrasse":"has_terrace","Garage":"has_garage","Parking":"has_parking",
    "Parking sous-sol":"has_underground_parking","Parking extérieur":"has_outdoor_parking",
    "Vue sur mer":"has_sea_view","Meublé":"is_furnished",
    "Cuisine équipée":"has_equipped_kitchen","Cuisine indépendante":"has_separate_kitchen",
    "Interphone":"has_intercom","Balcon":"has_balcony","Cave":"has_cellar",
    "Sécurité":"has_security","Gardien":"has_caretaker",
    "Caméra de sécurité":"has_cctv","Vidéosurveillance":"has_cctv",
    "Double vitrage":"has_double_glazing","Fibre optique":"has_fiber_optic",
    "Digicode":"has_digicode","Porte blindée":"has_armored_door",
    "Lumineux":"is_bright","Calme":"is_quiet",
}

os.makedirs(MEDIA_ROOT, exist_ok=True)
print("✅ Cellule 2 — Configuration chargée")
print(f"   {len(CATEGORIES)} catégories | TARGET={TARGET_PER_TYPE}/catégorie")
print(f"   {len(CITY_TO_GEO)} villes/délégations indexées")


✅ Cellule 2 — Configuration chargée
   8 catégories | TARGET=200/catégorie
   116 villes/délégations indexées


In [3]:
# ═══════════════════════════════════════════════════════════
# CELLULE 3 — BASE DE DONNÉES  (sans subtitle, sans price_raw)
# ═══════════════════════════════════════════════════════════

def get_conn():
    return psycopg2.connect(**DB_CONFIG)

def create_tables():
    conn = get_conn(); cur = conn.cursor()
    try:
        cur.execute("DROP TABLE IF EXISTS fi_dari_listings CASCADE;")
        cur.execute("DROP TABLE IF EXISTS fi_dari_log CASCADE;")
        cur.execute("""
            CREATE TABLE fi_dari_listings (
                id               SERIAL PRIMARY KEY,
                title            TEXT,
                price            REAL,
                transaction_type VARCHAR(20),
                type             VARCHAR(30),
                region           VARCHAR(100),
                municipality     VARCHAR(100),
                zone             VARCHAR(50),
                location_details JSONB,
                surface          REAL,
                rooms            INTEGER,
                description      TEXT,
                features         JSONB,
                poi              JSONB,
                image_path       TEXT,
                images_folder    TEXT,
                images_count     INTEGER DEFAULT 0,
                url              TEXT UNIQUE NOT NULL,
                pdf_link         TEXT,
                last_updated     TEXT,
                scraped_at       TIMESTAMP DEFAULT NOW(),
                is_new           BOOLEAN DEFAULT TRUE
            );
        """)
        for sql in [
            "CREATE INDEX idx_fd_type        ON fi_dari_listings(type);",
            "CREATE INDEX idx_fd_transaction ON fi_dari_listings(transaction_type);",
            "CREATE INDEX idx_fd_region      ON fi_dari_listings(region);",
            "CREATE INDEX idx_fd_zone        ON fi_dari_listings(zone);",
            "CREATE INDEX idx_fd_price       ON fi_dari_listings(price);",
            "CREATE INDEX idx_fd_scraped     ON fi_dari_listings(scraped_at);",
            "CREATE INDEX idx_fd_url         ON fi_dari_listings(url);",
            "CREATE INDEX idx_is_new         ON fi_dari_listings(is_new);",
        ]:
            cur.execute(sql)
        cur.execute("""
            CREATE TABLE fi_dari_log (
                id           SERIAL PRIMARY KEY,
                run_type     VARCHAR(20),
                started_at   TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
                finished_at  TIMESTAMP,
                total_new    INTEGER DEFAULT 0,
                total_errors INTEGER DEFAULT 0,
                status       VARCHAR(20) DEFAULT 'running'
            );
        """)
        conn.commit()
        print("✅ Tables créées (sans subtitle, sans price_raw).")
    except Exception as e:
        conn.rollback(); print(f"❌ Erreur SQL : {e}")
    finally:
        cur.close(); conn.close()

def url_exists(cur, url):
    cur.execute("SELECT 1 FROM fi_dari_listings WHERE url=%s LIMIT 1", (url,))
    return cur.fetchone() is not None

def insert_listing(cur, d):
    try:
        cur.execute("""
            INSERT INTO fi_dari_listings
                (title, price, transaction_type, type,
                 region, municipality, zone, location_details,
                 surface, rooms, features, poi, description,
                 url, pdf_link, image_path, images_folder,
                 images_count, last_updated, is_new)
            VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)
            ON CONFLICT(url) DO NOTHING RETURNING id
        """, (
            d.get("title"), d.get("price"),
            d.get("transaction_type"), d.get("type"),
            d.get("region"), d.get("municipality"), d.get("zone"),
            json.dumps(d.get("location_details", {}), ensure_ascii=False),
            d.get("surface"), d.get("rooms"),
            json.dumps(d.get("features", {}), ensure_ascii=False),
            json.dumps(d.get("poi",      {}), ensure_ascii=False),
            d.get("description", ""), d.get("url"), d.get("pdf_link"),
            None, None, 0, d.get("last_updated", "Unknown"), True,
        ))
        row = cur.fetchone()
        return row[0] if row else None
    except Exception as e:
        print(f"      ❌ Insert: {e}"); return None

def update_images(cur, listing_id, thumb, folder, count):
    cur.execute("""
        UPDATE fi_dari_listings
        SET image_path=%s, images_folder=%s, images_count=%s
        WHERE id=%s
    """, (thumb, folder, count, listing_id))

def log_start(cur, run_type):
    cur.execute("INSERT INTO fi_dari_log(run_type) VALUES(%s) RETURNING id", (run_type,))
    return cur.fetchone()[0]

def log_end(cur, log_id, n, e, status="done"):
    cur.execute("""
        UPDATE fi_dari_log
        SET finished_at=NOW(), total_new=%s, total_errors=%s, status=%s
        WHERE id=%s
    """, (n, e, status, log_id))

def reset_is_new(cur):
    cur.execute("UPDATE fi_dari_listings SET is_new=FALSE")

create_tables()


✅ Tables créées (sans subtitle, sans price_raw).


In [4]:
# ═══════════════════════════════════════════════════════════
# CELLULE 4 — UTILITAIRES
# Corrections :
#   - Localisation INTELLIGENTE (titre+desc+balises, pas "Tunisie" par défaut)
#   - Images : filtre strict URL + taille + dimensions
#   - Suppression subtitle partout
# ═══════════════════════════════════════════════════════════

import unicodedata

def _normalize(s):
    """Minuscule + supprime accents pour comparaison robuste."""
    return unicodedata.normalize("NFKD", s.lower()).encode("ascii", "ignore").decode()

# Index normalisé pour la recherche géo
_CITY_NORM = {_normalize(k): v for k, v in CITY_TO_GEO.items()}

def _find_city_in_text(text):
    """
    Cherche la première ville connue dans un texte quelconque.
    Retourne (zone, region, city_name) ou None.
    Tri par longueur décroissante pour préférer les noms longs.
    """
    if not text: return None
    t = _normalize(text)
    for city_norm in sorted(_CITY_NORM, key=len, reverse=True):
        pat = r'\b' + re.escape(city_norm) + r'\b'
        if re.search(pat, t):
            zone, region = _CITY_NORM[city_norm]
            orig = next(k for k in CITY_TO_GEO if _normalize(k) == city_norm)
            return zone, region, orig.title()
    return None

def _extract_municipality_clean(text, city_name):
    """
    Extrait la municipalité propre depuis un texte (1-3 mots max).
    Ex: "Achat appartement s+1 76m² Hammamet centre" → "Hammamet Centre"
    Ex: "Hammamet - Nabeul" → "Hammamet"
    Ex: "La Marsa, Ariana" → "La Marsa"
    """
    if not text or not city_name: return city_name
    t = _normalize(text)
    city_n = _normalize(city_name)
    # Chercher la ville dans le texte
    m = re.search(r'\b' + re.escape(city_n) + r'\b', t)
    if not m: return city_name
    # Prendre ce qui suit immédiatement la ville (qualificatif optionnel)
    after = t[m.end():m.end()+30].strip()
    # Qualifier autorisé : nord, sud, est, ouest, centre, ville, plage, corniche, etc.
    qualifiers = ['nord','sud','est','ouest','centre','ville','plage','corniche',
                  'centre ville','sud ouest','nord est','nord ouest','sud est',
                  'la marsa','el menzah','el mourouj','el ghazela','berges du lac',
                  'lac 1','lac 2','cite','cité']
    extra = ''
    for q in sorted(qualifiers, key=len, reverse=True):
        if after.startswith(q):
            extra = ' ' + q.title()
            break
    # Reconstuire avec casse propre
    result = city_name + extra
    return result.strip()

def build_location(raw_loc, title="", description="", soup=None):
    """
    Extraction géographique intelligente — 5 étapes par priorité :
    1. Balises HTML spécifiques (breadcrumb, localisation)
    2. raw_loc (texte brut de la page)
    3. Titre de l'annonce
    4. Description
    5. URL du bien (dernier recours)
    Retourne dict avec region, municipality, zone, location_details.
    """
    candidates = []

    # ── Étape 1 : balises HTML ─────────────────────────────────
    if soup:
        # Breadcrumb ou span localisation
        for tag in soup.find_all(["span","p","div","li","a"],
                                  class_=re.compile(r'local|address|ville|region|breadcrumb|lieu|where|city', re.I)):
            t = tag.get_text(" ", strip=True)
            if 2 < len(t) < 150: candidates.append(t)
        # Balises meta
        for meta in soup.find_all("meta"):
            prop = (meta.get("property","") + meta.get("name","")).lower()
            if any(k in prop for k in ["locality","city","region","address"]):
                c = meta.get("content","").strip()
                if c: candidates.append(c)

    # ── Étape 2 : raw_loc ─────────────────────────────────────
    if raw_loc and raw_loc.lower() not in ("tunisie", ""):
        candidates.insert(0, raw_loc)

    # ── Étape 3 & 4 : titre + description ────────────────────
    candidates.append(title)
    if description: candidates.append(description[:500])

    # ── Recherche dans chaque candidat ───────────────────────
    found = None
    municipality_raw = ""
    for cand in candidates:
        res = _find_city_in_text(cand)
        if res:
            zone, region, city_name = res
            found = (zone, region, city_name)
            # Extraire la municipalité propre (nom de ville + qualificatif max)
            municipality_raw = _extract_municipality_clean(cand, city_name)
            break

    if found:
        zone, region, city_name = found
        municipality = municipality_raw or city_name
        location_details = {
            "region": region, "municipality": municipality,
            "zone": zone, "raw": raw_loc or title
        }
        return {
            "region": region, "municipality": municipality,
            "zone": zone, "location_details": location_details
        }

    # ── Fallback : parse raw_loc basique ─────────────────────
    parts = [p.strip() for p in (raw_loc or "").replace(" - ", ",").split(",") if p.strip()]
    region_raw = parts[-1] if parts else "Tunisie"
    municipality_raw = parts[-2] if len(parts) >= 2 else (parts[0] if parts else "Tunisie")

    # Chercher le gouvernorat dans GOUVERNORAT_ZONE
    zone = "autre"
    for gov, z in GOUVERNORAT_ZONE.items():
        if _normalize(gov) in _normalize(region_raw):
            zone = z; break

    location_details = {
        "region": region_raw, "municipality": municipality_raw,
        "zone": zone, "raw": raw_loc or "Tunisie"
    }
    return {
        "region": region_raw, "municipality": municipality_raw,
        "zone": zone, "location_details": location_details
    }

def get_zone(region):
    """Fallback zone depuis nom de région."""
    r = _normalize(region)
    for gov, z in GOUVERNORAT_ZONE.items():
        if _normalize(gov) in r: return z
    for city_norm, (zone, _) in _CITY_NORM.items():
        if city_norm in r: return zone
    return "autre"

# ────────────────────────────────────────────────────────────
def extract_rooms(text):
    if not text: return None
    t = text.lower()
    for pat in [r'\bs\s*\+?\s*(\d)\b', r'\b[ft]\s*(\d)\b',
                r'(\d)\s*pi[eè]ces?', r'(\d)\s*chambres?', r'\bstudio\b']:
        m = re.search(pat, t)
        if m:
            if 'studio' in pat: return 1
            v = int(m.group(1))
            return v if 1 <= v <= 15 else None
    return None

def extract_surface(text):
    if not text: return None
    t = text.lower()
    for pat in [r'([\d][\d\s.,]*)\s*m\s*[²2]',
                r'(?:superficie|surface)\s*(?:de|:)?\s*([\d][\d.,]*)',
                r'de\s+([\d][\d.,]*)\s*m\s*[²2]']:
        m = re.search(pat, t)
        if m:
            try:
                v = float(m.group(1).replace(' ', '').replace(',', '.'))
                if 10 <= v <= 50000: return v
            except: continue
    return None

def extract_price(text):
    if not text: return None
    t = str(text)
    t = re.sub(r'(?i)\b(tnd|dt|dinar|dinars|euro|eur|usd|\$|€|/mois|/an)\b', '', t)
    t = t.replace('\xa0', '').replace('\u202f', '').replace('\u00a0', '')
    digits = re.sub(r'[^\d.,]', '', t.strip())
    if not digits: return None
    if re.match(r'^\d{1,3}[.,]\d{3}$', digits):
        digits = digits.replace('.', '').replace(',', '')
    elif digits.count('.') > 1 or digits.count(',') > 1:
        digits = digits.replace('.', '').replace(',', '')
    elif '.' in digits and ',' in digits:
        if digits.index('.') < digits.index(','):
            digits = digits.replace('.', '').replace(',', '.')
        else:
            digits = digits.replace(',', '')
    elif ',' in digits and '.' not in digits:
        parts = digits.split(',')
        if len(parts[1]) == 3:
            digits = digits.replace(',', '')
        else:
            digits = digits.replace(',', '.')
    try:
        v = float(digits)
        return v if 500 <= v <= 100_000_000 else None
    except:
        return None

def extract_floor(text):
    if not text: return None
    t = text.lower()
    for pat in [r'(\d+)[eè]me?\s*[eé]tage', r'\b(rdc|rez[- ]de[- ]chauss[eé]e)\b',
                r'\b(dernier\s*[eé]tage)\b', r'[eé]tage\s*(\d+)']:
        m = re.search(pat, t)
        if m: return m.group(0).strip()
    return None

def extract_poi_from_description(description):
    if not description: return {}
    poi = {}; t = description.lower()
    kws = {
        "école":        ["école","ecole","collège","lycée"],
        "université":   ["université","faculté"],
        "hôpital":      ["hôpital","hopital","clinique","polyclinique"],
        "pharmacie":    ["pharmacie"],
        "supermarché":  ["supermarché","superette","carrefour"],
        "transport":    ["bus","métro","train","gare","arrêt"],
        "mosquée":      ["mosquée","mosquee"],
        "plage":        ["plage","mer"],
        "centre ville": ["centre ville","centre-ville"],
        "aéroport":     ["aéroport","aeroport"],
    }
    dist_pats = [r'(?:à\s*)?(\d+)\s*m(?:ètres?)?',
                 r'(?:à\s*)?(\d+(?:[.,]\d+)?)\s*km',
                 r'(?:à\s*)?(\d+)\s*min(?:utes?)?',
                 r'\b(proche|à proximit[eé]|en face)']
    for name, words in kws.items():
        for kw in words:
            i = t.find(kw)
            if i == -1: continue
            ctx = t[max(0, i-20):i+60]; dist = None
            for dp in dist_pats:
                dm = re.search(dp, ctx)
                if dm: dist = dm.group(0).strip(); break
            poi[name] = dist if dist else "mentionné"; break
    return poi

# ── FILTRE IMAGES — PILLOW UNIQUEMENT ───────────────────────────────────────
# Philosophie : NE PAS filtrer par URL, domaine ou extension.
# fi-dari.tn sert les logos agences depuis son propre CDN (/static/media/)
# avec des URLs sans extension lisible (ex: logo-biat.2781cea3.webp).
# La SEULE distinction fiable : les dimensions de l'image.
#   - Logo agence     : carré ~200×200 px  → rejeté
#   - Photo de bien   : paysage ≥ 400×300 px → conservé
#
# Seules exclusions URL conservées :
#   - data: URLs (base64 inline)
#   - .svg / .gif / .ico  (jamais des photos)

# Extensions toujours rejetées (vecteurs/animations, jamais des photos réelles)
_SKIP_EXT = {".svg", ".gif", ".ico", ".bmp", ".tiff", ".woff", ".woff2",
             ".ttf", ".eot", ".js", ".css", ".json", ".xml", ".txt"}

def _skip_url(url: str) -> bool:
    """Rejette uniquement data: et extensions non-image."""
    if not url or url.startswith("data:"): return True
    u = url.lower().split("?")[0].split("#")[0]
    # Extensions JS/CSS/font
    if any(u.endswith(e) for e in _SKIP_EXT): return True
    return False

def _normalize_url(src: str) -> str:
    """Normalise une URL relative en absolue."""
    src = src.strip().strip("'").strip('"') 
    if src.startswith("//"): return "https:" + src
    if src.startswith("/"): return "https://fi-dari.tn" + src
    if not src.startswith("http"): return "https://fi-dari.tn/" + src.lstrip("/")
    return src

def _get_hd_url(url: str) -> str:
    """Essaie de récupérer la version HD en supprimant les suffixes miniature."""
    return re.sub(r'[_-](thumb|small|medium|xs|sm|md|150x150|300x300|640x|400x)',
                  '', url, flags=re.IGNORECASE)

def download_all_images(img_urls, prop_type, listing_id):
    """
    Télécharge toutes les images candidates et garde UNIQUEMENT
    celles dont Pillow confirme qu'elles sont assez grandes.

    SEUL FILTRE : dimensions ≥ MIN_W × MIN_H pixels.
      - Logo agence MS Immobilier : ~200×200  → rejeté
      - Photo de bien             : ≥ 400×300 → conservé

    Pas de filtre URL, pas de filtre domaine, pas de filtre extension.
    """
    MIN_W, MIN_H = 400, 300   # seuil : une vraie photo de bien est au moins 400×300

    folder = os.path.join(MEDIA_ROOT, prop_type, str(listing_id))
    os.makedirs(folder, exist_ok=True)
    if not img_urls: return None, folder, 0

    # Dédoublonner
    seen_urls = set()
    candidates = []
    for url in img_urls:
        url = _normalize_url(url)
        if url not in seen_urls:
            seen_urls.add(url)
            candidates.append(url)

    saved = []
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
                      "AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36",
        "Referer": "https://fi-dari.tn/",
        "Accept": "image/avif,image/webp,image/apng,image/*,*/*;q=0.8",
    }

    # Charger Pillow
    try:
        from PIL import Image as _PIL
        import io as _io
        has_pil = True
    except ImportError:
        has_pil = False
        print("            ⚠️  Pillow absent — filtre dimensions désactivé (pip install Pillow)")

    for url in candidates:
        if _skip_url(url): continue

        # Essayer d'abord la version HD
        url_hd = _get_hd_url(url)
        try_urls = [url_hd, url] if url_hd != url else [url]

        for try_url in try_urls:
            try:
                r = requests.get(try_url, headers=headers, timeout=15, stream=False)
                if r.status_code != 200: continue
                content = r.content
                if len(content) < 1000: continue   # trop petit pour être une image réelle

                if has_pil:
                    try:
                        img_obj = _PIL.open(_io.BytesIO(content))
                        w, h = img_obj.size
                        if w < MIN_W or h < MIN_H:
                            print(f"            ⏭  {w}×{h}px trop petit → {try_url[-55:]}")
                            break   # rejeter cette URL (pas la peine d'essayer l'original)
                    except Exception:
                        pass      # Si Pillow ne peut pas lire → garder quand même
                else:
                    # Sans Pillow : filtrer uniquement par taille fichier (logo ~6-15 KB)
                    if len(content) < 30_000:
                        print(f"            ⏭  {len(content)//1024}KB trop petit → {try_url[-55:]}")
                        break

                # Déterminer l'extension depuis Content-Type ou URL
                ct = r.headers.get("Content-Type", "")
                if "webp" in ct:   ext = "webp"
                elif "png"  in ct: ext = "png"
                elif "jpeg" in ct or "jpg" in ct: ext = "jpg"
                else:
                    tail = try_url.split("?")[0].split(".")[-1].lower()
                    ext  = tail if tail in ("jpg","jpeg","png","webp") else "jpg"

                num = len(saved) + 1
                fp  = os.path.join(folder, f"image_{num:03d}.{ext}")
                with open(fp, "wb") as fh:
                    fh.write(content)
                saved.append(fp)
                sz = len(content) // 1024
                print(f"            ✅ image_{num:03d}.{ext}  {w if has_pil else '??'}×{h if has_pil else '??'}px  {sz}KB")
                time.sleep(0.3)
                break   # succès → passer à l'image suivante

            except Exception as ex:
                print(f"            ❌ {ex} — {try_url[-60:]}")
                continue

    if not saved:
        print(f"            ⚠️  Aucune photo conservée pour listing {listing_id}")
    return (saved[0] if saved else None), folder, len(saved)

# ── FEATURES ────────────────────────────────────────────────────────────────
NAV_EXCLUDE = {
    "blog","accueil","youtube","facebook","instagram","tiktok","twitter",
    "linkedin","pinterest","se connecter","contactez-nous","contact",
    "fi-dari.tn","fi-dari","fidari","voir prix","immobilier neuf",
    "immobilier ancien","terrain","bureau","maison","appartement",
    "vendre","louer","acheter","recherche","résultats","page","suivant",
    "précédent","retour","partager","imprimer","signaler","biatimmo",
    "agence","promoteur","annonceur","newsletter","abonnez","inscription",
    "connexion","créer un compte","mot de passe","mes annonces","mes favoris",
}
PRICE_IN_TEXT = re.compile(r'\d[\d\s]*(?:000)?\s*(?:dt|tnd|dinar|€|\$)', re.IGNORECASE)

def map_features(raw_list, description=""):
    result = {}
    for item in raw_list:
        item_clean = item.strip()
        if not item_clean or len(item_clean) > 120: continue
        item_lower = item_clean.lower()
        if any(ex in item_lower for ex in NAV_EXCLUDE): continue
        if PRICE_IN_TEXT.search(item_clean): continue
        if "http" in item_lower or "www." in item_lower or ".tn" in item_lower: continue
        if re.match(r'^\d+$', item_clean): continue
        mapped = False
        for fr, en in FEATURE_MAPPING.items():
            if fr.lower() in item_lower:
                result[en] = True; mapped = True; break
        if not mapped:
            result[item_clean] = True
    floor = extract_floor(" ".join(raw_list) + " " + description)
    if floor: result["floor"] = floor
    return result

print("✅ Cellule 4 — Utilitaires chargés")
print(f"   Géolocalisation intelligente : {len(_CITY_NORM)} villes indexées")
print("   Filtre images : URL + extension + taille + dimensions")


✅ Cellule 4 — Utilitaires chargés
   Géolocalisation intelligente : 112 villes indexées
   Filtre images : URL + extension + taille + dimensions


In [5]:
# ═══════════════════════════════════════════════════════════
# CELLULE 5 — SELENIUM + SCRAPING
# ═══════════════════════════════════════════════════════════

_RE_PRICE   = re.compile(r'price|prix|montant|tarif|cost',    re.IGNORECASE)
_RE_LOC     = re.compile(r'location|address|region|ville|localisation|adresse|local|lieu|city|where', re.IGNORECASE)
_RE_DATE    = re.compile(r'date|update|publication|posted|publi|ajout|modifi', re.IGNORECASE)
_RE_DESC    = re.compile(r'description|detail|content|text|body|info|propos',  re.IGNORECASE)
_RE_FEAT    = re.compile(r'feature|spec|equipement|caracteristique|attribute|tag|badge|chip|atout|avantage|prive|option', re.IGNORECASE)
_RE_GALLERY = re.compile(r'gallery|slider|carousel|photo|image|swiper|lightbox|media|galerie', re.IGNORECASE)
_RE_PRICE_TEXT = re.compile(
    r'(?:à\s+partir\s+de\s+)?([\d][\d\s.,]*)[\s\u00a0\u202f]*(?:dt|tnd|dinar)',
    re.IGNORECASE
)

def create_driver(headless=True):
    opts = Options()
    if headless: opts.add_argument("--headless=new")
    opts.add_argument("--no-sandbox")
    opts.add_argument("--disable-dev-shm-usage")
    opts.add_argument("--window-size=1920,1080")
    opts.add_argument("--disable-blink-features=AutomationControlled")
    opts.add_experimental_option("excludeSwitches", ["enable-automation"])
    opts.add_experimental_option("useAutomationExtension", False)
    opts.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
                      "AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36")
    opts.set_capability("goog:loggingPrefs", {"performance": "ALL"})
    d = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=opts)
    d.execute_script("Object.defineProperty(navigator,'webdriver',{get:()=>undefined})")
    d.set_page_load_timeout(30)
    return d

def restart_driver(driver, headless=True):
    try: driver.quit()
    except: pass
    print("      🔄 Redémarrage Chrome...")
    time.sleep(5)
    return create_driver(headless=headless)

def build_url(slug, page_num):
    base = re.sub(r'&page=\d+', '', f"{BASE_URL}/{slug}")
    return f"{base}&page={page_num}"

def deep_scroll_and_wait(driver, wait_sec=10):
    time.sleep(wait_sec)
    total = driver.execute_script("return document.body.scrollHeight")
    step  = max(200, total // 15); pos = 0
    while pos < total:
        driver.execute_script(f"window.scrollTo(0, {pos});")
        time.sleep(0.3); pos += step
        new_h = driver.execute_script("return document.body.scrollHeight")
        if new_h > total: total = new_h
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(2)
    driver.execute_script("window.scrollTo(0, 0);")
    time.sleep(1)

def get_network_requests(driver):
    api_data = []
    for entry in driver.get_log("performance"):
        try:
            msg = json.loads(entry["message"])["message"]
            if msg["method"] != "Network.responseReceived": continue
            resp = msg["params"].get("response", {})
            url  = resp.get("url", ""); mime = resp.get("mimeType", "")
            if "json" in mime and any(k in url.lower() for k in
                    ["annonce","search","listing","bien","property","immobil","api"]):
                api_data.append({"url": url, "requestId": msg["params"].get("requestId", "")})
        except: continue
    return api_data

def fetch_request_body(driver, request_id):
    try:
        r    = driver.execute_cdp_cmd("Network.getResponseBody", {"requestId": request_id})
        body = r.get("body", "")
        if r.get("base64Encoded", False):
            import base64
            body = base64.b64decode(body).decode("utf-8", errors="ignore")
        return json.loads(body)
    except: return None

def get_annonces_from_list_page(driver, url):
    print(f"      🌐 Chargement...")
    driver.get(url)
    deep_scroll_and_wait(driver, wait_sec=10)

    for call in get_network_requests(driver):
        data = fetch_request_body(driver, call["requestId"])
        if data:
            ann = extract_from_api_response(data)
            if ann: print(f"      ✅ API JSON : {len(ann)} annonces"); return ann

    soup = BeautifulSoup(driver.page_source, "html.parser")
    urls = set()
    for a in soup.find_all("a", href=True):
        href = a["href"]
        if any(p in href for p in ["/bien/", "/annonce/", "/detail/"]):
            if not href.startswith("http"): href = BASE_URL + href
            urls.add(href)
    if urls:
        print(f"      ✅ Liens HTML : {len(urls)} annonces")
        return [{"url": u, "title_hint": "", "price_hint": "", "location_hint": "", "thumb": None}
                for u in urls]

    items = find_listing_items(soup)
    result, seen = [], set()
    for item in items:
        a = item.find("a", href=True)
        if not a: continue
        href = a["href"]
        if not href.startswith("http"): href = BASE_URL + href
        if href in seen: continue
        seen.add(href); result.append(extract_card_data(item, href))
    if result:
        print(f"      ✅ CSS heuristique : {len(result)} annonces")
        return result

    print(f"      ❌ Aucune annonce")
    with open(f"debug_{int(time.time())}.html", "w", encoding="utf-8") as f:
        f.write(driver.page_source)
    return []

def extract_from_api_response(data):
    annonces = []
    def find_list(obj, depth=0):
        if depth > 5: return []
        if isinstance(obj, list) and len(obj) > 1:
            s = obj[0]
            if isinstance(s, dict) and any(k in s for k in
                    ["url","price","prix","titre","title","reference","ref"]): return obj
        if isinstance(obj, dict):
            for v in obj.values():
                r = find_list(v, depth + 1)
                if r: return r
        return []
    for item in find_list(data):
        if not isinstance(item, dict): continue
        url = (item.get("url") or item.get("link") or item.get("slug") or "")
        if url and not url.startswith("http"): url = BASE_URL + "/bien/" + str(url)
        if not url: continue
        loc_hint = str(item.get("localisation","") or item.get("location","") or
                       item.get("ville","") or item.get("gouvernorat","") or "")
        annonces.append({
            "url":           url,
            "title_hint":    str(item.get("titre","") or item.get("title","") or ""),
            "price_hint":    str(item.get("prix","")  or item.get("price","") or ""),
            "location_hint": loc_hint,
            "thumb":         item.get("photo") or item.get("image"),
            "raw_data":      item,
        })
    return annonces

def find_listing_items(soup):
    cc = Counter()
    for tag in soup.find_all(["div","article","li","a"]):
        for cls in tag.get("class",[]): cc[cls] += 1
    best = []
    for cls, cnt in cc.items():
        if not (4 <= cnt <= 100): continue
        items = soup.find_all(attrs={"class": re.compile(re.escape(cls))})
        valid = [i for i in items if i.find("img") and i.find("a",href=True)
                 and len(i.get_text(strip=True)) > 20]
        if len(valid) > len(best): best = valid
    return best

def extract_card_data(item, url):
    title = ""
    for sel in ["h1","h2","h3","h4"]:
        el = item.find(sel)
        if el:
            t = el.get_text(strip=True)
            if 5 < len(t) < 200: title = t; break
    price_hint = ""
    for el in item.find_all(class_=_RE_PRICE):
        t = el.get_text(strip=True).replace("\xa0","").replace("\u202f","")
        if any(c.isdigit() for c in t): price_hint = t; break
    loc = ""
    for el in item.find_all(class_=_RE_LOC):
        t = el.get_text(strip=True)
        if 2 < len(t) < 150: loc = t; break
    thumb = None
    img = item.find("img")
    if img:
        thumb = (img.get("data-src") or img.get("data-lazy") or
                 img.get("data-original") or img.get("src",""))
        if thumb and not thumb.startswith("http"):
            thumb = BASE_URL + "/" + thumb.lstrip("/")
        if thumb and _is_parasite_url(thumb): thumb = None
    return {"url":url,"title_hint":title,"price_hint":price_hint,
            "location_hint":loc,"thumb":thumb}

def extract_price_fidari(soup):
    for tag in soup.find_all(class_=_RE_PRICE):
        t = tag.get_text(strip=True).replace("\xa0","").replace("\u202f","")
        v = extract_price(t)
        if v: return v
    for meta in soup.find_all("meta"):
        prop = (meta.get("property","") + meta.get("name","")).lower()
        if "price" in prop or "prix" in prop:
            v = extract_price(meta.get("content",""))
            if v: return v
    m = _RE_PRICE_TEXT.search(soup.get_text(" "))
    if m:
        v = extract_price(m.group(1))
        if v: return v
    h1 = soup.find("h1")
    if h1:
        v = extract_price(h1.get_text())
        if v: return v
    return None

def extract_date_fidari(soup):
    for tag in soup.find_all("time"):
        dt = tag.get("datetime","")
        if dt: return dt
        t = tag.get_text(strip=True)
        if len(t) > 3: return t
    for tag in soup.find_all(class_=_RE_DATE):
        t = tag.get_text(strip=True)
        if 3 < len(t) < 80: return t
    text = soup.get_text(" ", strip=True)
    for pat in [
        r'(?:modifi[eé]e?|publi[eé]e?|ajout[eé]e?|mise?\s+à\s+jour)\s*(?:le\s*)?(\d{1,2}[\s/\-\.]\w+[\s/\-\.]\d{2,4})',
        r'(il\s+y\s+a\s+\d+\s*(?:jour|heure|minute|mois|an)s?)',
        r'\b(\d{1,2}[/\-\.]\d{1,2}[/\-\.]\d{4})\b',
        r'\b(\d{1,2}\s+(?:janvier|février|mars|avril|mai|juin|juillet|août|septembre|octobre|novembre|décembre)\s+\d{4})\b',
    ]:
        m = re.search(pat, text, re.IGNORECASE)
        if m: return m.group(1).strip()
    return "Unknown"

def extract_pdf_fidari(soup):
    for a in soup.find_all("a", href=True):
        href = a["href"].strip()
        text = a.get_text(strip=True).lower()
        href_low = href.lower()
        if href_low.endswith(".pdf"):
            return href if href.startswith("http") else BASE_URL + "/" + href.lstrip("/")
        if any(kw in text for kw in ["fiche","télécharger","download","pdf","brochure"]):
            return href if href.startswith("http") else BASE_URL + "/" + href.lstrip("/")
        if any(kw in href_low for kw in [".pdf","fiche","download","telecharger","brochure"]):
            return href if href.startswith("http") else BASE_URL + "/" + href.lstrip("/")
    return None

def extract_location_from_page(soup, driver_url=""):
    """
    Cherche la localisation dans les balises HTML de la page de détail.
    Retourne le texte brut le plus pertinent.
    """
    # 1. Breadcrumb / fil d'Ariane
    for tag in soup.find_all(["nav","ol","ul"], class_=re.compile(r'breadcrumb|fil|ariane|path', re.I)):
        items = tag.find_all(["li","a","span"])
        parts = [i.get_text(strip=True) for i in items if i.get_text(strip=True)]
        if len(parts) >= 2:
            # Exclure le premier (Accueil) et le dernier (titre annonce)
            geo_parts = [p for p in parts[1:-1] if len(p) > 2 and len(p) < 60]
            if geo_parts: return " - ".join(geo_parts)
    # 2. Span/div localisation
    for tag in soup.find_all(class_=_RE_LOC):
        t = tag.get_text(" ", strip=True)
        if 3 < len(t) < 150: return t
    # 3. Icône localisation (souvent un i ou svg suivi du texte)
    for tag in soup.find_all(["p","span","div"]):
        text = tag.get_text(" ", strip=True)
        if any(kw in text.lower() for kw in ["tunisie","tunis","sfax","sousse","nabeul",
                "monastir","bizerte","gabes","kairouan","sousse","mahdia"]):
            if 3 < len(text) < 150: return text
    # 4. URL : extraire depuis le slug /bien/ville-...
    m = re.search(r'/bien/([^/?#]+)', driver_url)
    if m:
        slug = m.group(1).replace("-", " ")
        return slug
    return ""

def collect_image_urls(soup, driver):
    """
    Collecte TOUTES les URLs d'images de la page sans filtrage URL.
    Le filtrage se fait UNIQUEMENT dans download_all_images() par dimensions Pillow.

    Sources (par ordre de fiabilité) :
    1. Log réseau Chrome  → toutes les images réellement chargées par le navigateur
    2. Attributs <img>    → data-big / data-original / data-src / src
    3. Background CSS     → url(...) dans style=""
    """
    seen   = set()
    result = []

    def add(src):
        if not src: return
        src = src.strip().strip("'").strip('"') 
        if not src or src.startswith("data:"): return
        if src.startswith("//"): src = "https:" + src
        if src.startswith("/"): src = "https://fi-dari.tn" + src
        if not src.startswith("http"): src = "https://fi-dari.tn/" + src.lstrip("/")
        # Exclure uniquement les ressources non-image évidentes
        u = src.lower().split("?")[0]
        if any(u.endswith(e) for e in (".svg",".gif",".ico",".js",".css",
                                        ".woff",".woff2",".ttf",".json")): return
        if src not in seen:
            seen.add(src)
            result.append(src)

    # ── 1. Log réseau Chrome ─────────────────────────────────
    # React charge les images via XHR/fetch → elles apparaissent dans les logs
    # C'est la source la plus fiable : on voit exactement ce qui s'affiche
    net_count = 0
    for entry in driver.get_log("performance"):
        try:
            msg = json.loads(entry["message"])["message"]
            # Requêtes réseau sortantes
            if msg["method"] == "Network.requestWillBeSent":
                req_url = msg["params"]["request"]["url"]
                u_low = req_url.lower()
                # Garder uniquement les ressources image (par Content-Type ou extension)
                if any(e in u_low for e in (".jpg",".jpeg",".png",".webp",
                                             "image/","media/","upload","static")):
                    add(req_url)
                    net_count += 1
        except: continue

    # ── 2. Attributs <img> ───────────────────────────────────
    for img in soup.find_all("img"):
        for attr in ["data-big","data-original","data-full",
                     "data-src","data-lazy","data-zoom-image","src"]:
            val = img.get(attr, "")
            if val: add(val); break

    # ── 3. Background CSS ────────────────────────────────────
    for el in soup.find_all(style=True):
        for m in re.finditer(r"url\(([^)]+)\)", el.get("style", "")):
            src_css = m.group(1).strip().strip("'").strip('"')
            add(src_css)

    print(f"            📦 {len(result)} URLs candidates ({net_count} réseau, "
          f"{len(result)-net_count} HTML)")
    return result


def scrape_detail_page(driver, url):
    try:
        driver.get(url)
        deep_scroll_and_wait(driver, wait_sec=7)
        soup = BeautifulSoup(driver.page_source, "html.parser")
        data = {}

        h1 = soup.find("h1")
        if h1 and len(h1.get_text(strip=True)) > 5:
            data["title_detail"] = h1.get_text(strip=True)

        desc = ""; best_len = 0
        for tag in soup.find_all(["div","p","section"]):
            cs = " ".join(tag.get("class",[]))
            if _RE_DESC.search(cs):
                t = tag.get_text(" ", strip=True)
                if 30 < len(t) < 5000 and len(t) > best_len: best_len = len(t); desc = t
        data["description"] = desc

        data["price"]        = extract_price_fidari(soup)
        data["last_updated"] = extract_date_fidari(soup)
        data["pdf_link"]     = extract_pdf_fidari(soup)

        # Localisation enrichie
        loc_raw = extract_location_from_page(soup, driver_url=url)
        data["location_raw"] = loc_raw

        raw_feats = []
        for tag in soup.find_all(class_=_RE_FEAT):
            t = tag.get_text(strip=True)
            if 2 < len(t) < 100: raw_feats.append(t)
        for ul in soup.find_all("ul"):
            lis = ul.find_all("li")
            if 2 <= len(lis) <= 30:
                for li in lis:
                    t = li.get_text(strip=True)
                    if 2 < len(t) < 80: raw_feats.append(t)
        raw_feats = list(dict.fromkeys(raw_feats))
        specs     = " ".join(raw_feats)

        data["surface_detail"] = extract_surface(specs) or extract_surface(desc)
        data["rooms_detail"]   = extract_rooms(specs)   or extract_rooms(desc)
        data["features"]       = map_features(raw_feats, desc)
        data["poi"]            = extract_poi_from_description(desc)
        data["img_urls"]       = collect_image_urls(soup, driver)
        data["_soup"]          = soup  # transmis pour build_location

        print(f"         🖼️  {len(data['img_urls'])} photos | 💰 {data['price']} | 📅 {data['last_updated']} | 📍 {loc_raw[:40]}")
        return data
    except Exception as e:
        print(f"      ⚠️ scrape_detail: {e}")
        return {}

print("✅ Cellule 5 — Selenium chargé")


✅ Cellule 5 — Selenium chargé


In [6]:
# ═══════════════════════════════════════════════════════════
# CELLULE 6 — DIAGNOSTIC VISUEL
# ═══════════════════════════════════════════════════════════

def run_diagnostic():
    test_url = (f"{BASE_URL}/search?objectif=vendre"
                f"&categorie=Appartement&usage=Habitation&bounds={B}&page=1")
    print("🔍 DIAGNOSTIC fi-dari.tn\n")
    d = create_driver(headless=False)
    try:
        d.get(test_url)
        print("   ⏳ Attente React (15s)...")
        deep_scroll_and_wait(d, wait_sec=12)
        soup = BeautifulSoup(d.page_source, "html.parser")
        print(f"   📑 Titre : {soup.title.string if soup.title else 'N/A'}")
        print(f"   📏 HTML  : {len(d.page_source):,} chars")
        liens = [a["href"] for a in soup.find_all("a", href=True)
                 if any(p in a["href"] for p in ["/bien/","/annonce/","/detail/"])]
        print(f"\n   🔗 Annonces trouvées : {len(liens)}")
        for l in liens[:5]: print(f"      → {l[:90]}")
        if liens:
            test_detail = liens[0] if liens[0].startswith("http") else BASE_URL + liens[0]
            print(f"\n   🧪 Test détail : {test_detail}")
            d.get(test_detail)
            deep_scroll_and_wait(d, wait_sec=8)
            soup2 = BeautifulSoup(d.page_source, "html.parser")
            loc_raw = extract_location_from_page(soup2, driver_url=test_detail)
            geo     = build_location(loc_raw,
                                     title=soup2.find("h1").get_text() if soup2.find("h1") else "",
                                     soup=soup2)
            imgs    = collect_image_urls(soup2, d)
            print(f"      💰 Prix       : {extract_price_fidari(soup2)}")
            print(f"      📅 Date       : {extract_date_fidari(soup2)}")
            print(f"      📄 PDF        : {extract_pdf_fidari(soup2)}")
            print(f"      📍 Loc brute  : {loc_raw[:60]}")
            print(f"      🗺️  Region     : {geo['region']}")
            print(f"      🏙️  Municipalité: {geo['municipality']}")
            print(f"      🌍 Zone       : {geo['zone']}")
            print(f"      🖼️  Images     : {len(imgs)}")
            for u in imgs[:3]: print(f"         {u[:90]}")
        with open("fidari_diagnostic.html","w",encoding="utf-8") as f:
            f.write(d.page_source)
        print("\n   💾 fidari_diagnostic.html sauvegardé")
    finally:
        input("\n   ⏸️  Appuie ENTRÉE pour fermer Chrome...")
        d.quit()

run_diagnostic()


🔍 DIAGNOSTIC fi-dari.tn

   ⏳ Attente React (15s)...
   📑 Titre : Rechercher un bien immobilier en Tunisie
   📏 HTML  : 47,479 chars

   🔗 Annonces trouvées : 10
      → /bien/Yesmine hammamet-PA32031-1
      → /bien/Hammemet centre-PA32022-1
      → /bien/Hammemet centre-PA32018-1
      → /bien/Hammemet centre-PA31994-1
      → /bien/Hammemet centre-PA31992-1

   🧪 Test détail : https://fi-dari.tn/bien/Yesmine hammamet-PA32031-1
            📦 47 URLs candidates (64 réseau, -17 HTML)
      💰 Prix       : 360000.0
      📅 Date       : 19 février 2026
      📄 PDF        : None
      📍 Loc brute  : Achat appartement s+1 de 90m² Yesmine hammamet, Nabeul Simul
      🗺️  Region     : Nabeul
      🏙️  Municipalité: Hammamet
      🌍 Zone       : nord
      🖼️  Images     : 47
         https://fi-dari.tn/static/media/logo-fidari.938f7b7a.webp
         https://b.tile.openstreetmap.fr/osmfr/6/33/25.png
         https://c.tile.openstreetmap.fr/osmfr/6/34/25.png

   💾 fidari_diagnostic.html sauvega


   ⏸️  Appuie ENTRÉE pour fermer Chrome... 


In [ ]:
# ═══════════════════════════════════════════════════════════
# CELLULE 7 — SCRAPING INITIAL (lancer UNE SEULE FOIS)
# ═══════════════════════════════════════════════════════════

conn   = get_conn()
cur    = conn.cursor()
driver = create_driver(headless=True)

log_id       = log_start(cur, "initial")
conn.commit()
total_saved  = 0
total_errors = 0
stats        = {}

print("🚀 SCRAPING INITIAL Fi-Dari")
print(f"   Objectif  : {TARGET_PER_TYPE} par catégorie")
print(f"   Total visé: {TARGET_PER_TYPE * len(CATEGORIES)} annonces\n")

try:
    for slug, (trans_type, prop_type) in CATEGORIES.items():
        cat_key   = f"{prop_type}_{trans_type}"
        cat_saved = 0
        print(f"\n{'═'*65}")
        print(f"  📦  {prop_type.upper()} | {trans_type.upper()}")
        print(f"{'═'*65}")

        for page_num in range(1, MAX_PAGES_INITIAL + 1):
            if cat_saved >= TARGET_PER_TYPE:
                print(f"  🎯 Objectif {TARGET_PER_TYPE} atteint."); break

            url_page      = build_url(slug, page_num)
            print(f"\n  📄 Page {page_num}")
            page_annonces = []
            for attempt in range(3):
                try:
                    page_annonces = get_annonces_from_list_page(driver, url_page); break
                except Exception as e:
                    print(f"      ⚠️ Retry {attempt+1}/3: {e}")
                    driver = restart_driver(driver); time.sleep(random.uniform(5,10))

            if not page_annonces:
                print("  🛑 Fin de catégorie."); break

            print(f"      📋 {len(page_annonces)} annonces")

            for ann in page_annonces:
                if cat_saved >= TARGET_PER_TYPE: break
                ann_url = ann.get("url","")
                if not ann_url or url_exists(cur, ann_url): continue

                print(f"      ⬇️  [{cat_saved+1}/{TARGET_PER_TYPE}] {ann_url[-60:]}")

                det = {}
                for attempt in range(3):
                    try: det = scrape_detail_page(driver, ann_url); break
                    except Exception as e:
                        print(f"      ⚠️ Retry détail {attempt+1}/3: {e}")
                        driver = restart_driver(driver); time.sleep(random.uniform(5,10))

                title   = det.get("title_detail") or ann.get("title_hint","") or "N/A"
                price   = det.get("price") or extract_price(ann.get("price_hint",""))
                loc_raw = det.get("location_raw","") or ann.get("location_hint","")
                desc    = det.get("description","")
                soup_   = det.get("_soup")
                cfg     = PROPERTY_CONFIG.get(prop_type, {"has_rooms":True,"has_surface":True})

                # ── Géolocalisation intelligente ──────────────────────────
                geo = build_location(
                    raw_loc     = loc_raw,
                    title       = title,
                    description = desc,
                    soup        = soup_
                )

                surface = (det.get("surface_detail") or extract_surface(title) or
                           extract_surface(desc)) if cfg["has_surface"] else None
                rooms   = (det.get("rooms_detail") or extract_rooms(title) or
                           extract_rooms(desc)) if cfg["has_rooms"] else None

                data = {
                    "title": title, "price": price,
                    "transaction_type": trans_type, "type": prop_type,
                    "region": geo["region"], "municipality": geo["municipality"],
                    "zone": geo["zone"], "location_details": geo["location_details"],
                    "surface": surface, "rooms": rooms,
                    "features": det.get("features",{}), "poi": det.get("poi",{}),
                    "description": desc, "url": ann_url,
                    "pdf_link": det.get("pdf_link"),
                    "last_updated": det.get("last_updated","Unknown"),
                }

                listing_id = insert_listing(cur, data)
                if not listing_id: total_errors += 1; continue
                conn.commit()

                img_urls = list(det.get("img_urls",[]))
                if ann.get("thumb") and ann["thumb"] not in img_urls:
                    if not _is_parasite_url(ann["thumb"]) and _is_photo_url(ann["thumb"]):
                        img_urls.insert(0, ann["thumb"])

                thumb, folder, nb = download_all_images(img_urls, prop_type, listing_id)
                update_images(cur, listing_id, thumb, folder, nb)
                conn.commit()

                cat_saved   += 1
                total_saved += 1
                print(f"         ✅ id={listing_id} | {geo['municipality']},{geo['region']} [{geo['zone']}] | prix={price} | {surface}m² | {nb} imgs")
                time.sleep(random.uniform(2,4))

            time.sleep(random.uniform(3,6))

        stats[cat_key] = cat_saved
        print(f"\n  📊 {cat_key} → {cat_saved} annonces")

except KeyboardInterrupt:
    print("\n⚠️ Interrompu.")

finally:
    log_end(cur, log_id, total_saved, total_errors)
    conn.commit()
    try: driver.quit()
    except: pass
    cur.close(); conn.close()
    print(f"\n{'═'*65}")
    print(f"  🎉 TERMINÉ — {total_saved} annonces | {total_errors} erreurs")
    print(f"{'═'*65}")
    for k, v in stats.items():
        print(f"  {k:<35} : {v:>4}")


🚀 SCRAPING INITIAL Fi-Dari
   Objectif  : 200 par catégorie
   Total visé: 1600 annonces


═════════════════════════════════════════════════════════════════
  📦  APARTMENT | SALE
═════════════════════════════════════════════════════════════════

  📄 Page 1
      🌐 Chargement...
      ✅ Liens HTML : 10 annonces
      📋 10 annonces
      ⬇️  [1/200] ge=Habitation&bounds=[[37.649,7.778],[30.107,11.953]]&page=1


In [9]:
# ═══════════════════════════════════════════════════════════
# CELLULE 8 — GÉNÉRATION daily_scrape.py + lancer_daily.bat
# ═══════════════════════════════════════════════════════════

import sys as _sys
import os

PROJECT_DIR = r"B:\EstateMind\fi_dari_scraper"
os.makedirs(os.path.join(PROJECT_DIR, "logs"), exist_ok=True)
DAILY_PY   = os.path.join(PROJECT_DIR, "daily_scrape.py")
BAT_FILE   = os.path.join(PROJECT_DIR, "lancer_daily.bat")
PYTHON_EXE = _sys.executable

DAILY_CONTENT = r"""# daily_scrape.py — Cycle quotidien Fi-Dari
import os, re, time, json, random, requests, psycopg2, unicodedata
from bs4 import BeautifulSoup
from datetime import datetime
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager

DB_CONFIG={"host":"localhost","database":"estate_mind_db","user":"postgres","password":"admin"}
MEDIA_ROOT=r"C:\EstateMind\media_fidari"
BASE_URL="https://fi-dari.tn"
B="[[37.649,7.778],[30.107,11.953]]"
MAX_PAGES_DAILY=3

CATEGORIES={
    f"search?objectif=vendre&categorie=Appartement&usage=Habitation&bounds={B}":("sale","apartment"),
    f"search?objectif=louer&categorie=Appartement&usage=Tout+type+de+location&bounds={B}":("rent","apartment"),
    f"search?objectif=vendre&categorie=Maison&usage=Habitation&bounds={B}":("sale","house"),
    f"search?objectif=louer&categorie=Maison&usage=Tout+type+de+location&bounds={B}":("rent","house"),
    f"search?objectif=vendre&categorie=Terrain&bounds={B}":("sale","land"),
    f"search?objectif=louer&categorie=Terrain&bounds={B}":("rent","land"),
    f"search?objectif=vendre&categorie=Bureau&usage=Professionnels&bounds={B}":("sale","office"),
    f"search?objectif=louer&categorie=Bureau&usage=Bureaux+et+commerce&bounds={B}":("rent","office"),
}

PROPERTY_CONFIG={
    'apartment':{'has_rooms':True,'has_surface':True},
    'house':{'has_rooms':True,'has_surface':True},
    'land':{'has_rooms':False,'has_surface':True},
    'office':{'has_rooms':False,'has_surface':True}
}

GOUVERNORAT_ZONE={
    'Tunis':'grand_tunis','Ariana':'grand_tunis','Ben Arous':'grand_tunis','Manouba':'grand_tunis',
    'Bizerte':'nord','Nabeul':'nord','Beja':'nord','Jendouba':'nord','Zaghouan':'nord',
    'Sousse':'sahel','Monastir':'sahel','Mahdia':'sahel',
    'Kairouan':'centre','Kasserine':'centre','Sidi Bouzid':'centre','Siliana':'centre','Gafsa':'centre',
    'Sfax':'sud','Gabes':'sud','Medenine':'sud','Tataouine':'sud','Tozeur':'sud','Kebili':'sud'
}

FEATURE_MAPPING={
    'Climatisation':'has_air_conditioning','Ascenseur':'has_elevator','Jardin':'has_garden',
    'Piscine':'has_pool','Terrasse':'has_terrace','Parking':'has_parking',
    'Vue sur mer':'has_sea_view','Meuble':'is_furnished','Balcon':'has_balcony',
    'Digicode':'has_digicode','Lumineux':'is_bright','Calme':'is_quiet'
}

_PARASITE_KW=[
    'biatimmo','fleximmo','logo','icon','/icons/','picto','sprite','favicon','avatar',
    'placeholder','loading','noimage','no-image','banner','ads/','agence','promoteur',
    'facebook','instagram','youtube','google','gstatic','map-marker','check','arrow','btn-',
    '.svg','.gif','.ico'
]
_PHOTO_EXT={'.jpg','.jpeg','.png','.webp'}

def _norm(s):
    return unicodedata.normalize('NFKD',s.lower()).encode('ascii','ignore').decode()

CITY_GEO={_norm(k):v for k,v in {
    'tunis':('grand_tunis','Tunis'),'ariana':('grand_tunis','Ariana'),
    'ben arous':('grand_tunis','Ben Arous'),'manouba':('grand_tunis','Manouba'),
    'la marsa':('grand_tunis','Tunis'),'marsa':('grand_tunis','Tunis'),
    'bardo':('grand_tunis','Tunis'),'el menzah':('grand_tunis','Ariana'),
    'ennasr':('grand_tunis','Ariana'),'soukra':('grand_tunis','Ariana'),
    'nabeul':('nord','Nabeul'),'hammamet':('nord','Nabeul'),'bizerte':('nord','Bizerte'),
    'jendouba':('nord','Jendouba'),'beja':('nord','Beja'),'zaghouan':('nord','Zaghouan'),
    'kelibia':('nord','Nabeul'),'korba':('nord','Nabeul'),
    'sousse':('sahel','Sousse'),'monastir':('sahel','Monastir'),'mahdia':('sahel','Mahdia'),
    'msaken':('sahel','Sousse'),'kantaoui':('sahel','Sousse'),
    'sfax':('sud','Sfax'),'gabes':('sud','Gabes'),'jerba':('sud','Medenine'),
    'djerba':('sud','Medenine'),'medenine':('sud','Medenine'),'tataouine':('sud','Tataouine'),
    'tozeur':('sud','Tozeur'),'kebili':('sud','Kebili'),'zarzis':('sud','Medenine'),
    'gafsa':('centre','Gafsa'),'kairouan':('centre','Kairouan'),'kasserine':('centre','Kasserine'),
    'sidi bouzid':('centre','Sidi Bouzid'),'siliana':('centre','Siliana')
}.items()}

def _is_parasite(url):
    u=url.lower()
    ext=os.path.splitext(u.split('?')[0])[1]
    if ext in ('.svg','.gif','.ico','.bmp'):
        return True
    return any(kw in u for kw in _PARASITE_KW)

def _is_photo(url):
    u=url.lower().split('?')[0]
    return any(u.endswith(e) for e in _PHOTO_EXT)

def _find_city(text):
    if not text:
        return None
    t=_norm(text)
    for city in sorted(CITY_GEO,key=len,reverse=True):
        if re.search(r'\b'+re.escape(city)+r'\b',t):
            zone,region=CITY_GEO[city]
            return zone,region,city.title()
    return None

def build_location(raw,title='',desc=''):
    for txt in [raw,title,desc[:300]]:
        r=_find_city(txt)
        if r:
            zone,region,city=r
            mun=city
            break
    else:
        region='Tunisie'; mun='Tunisie'; zone='autre'
    parts=[p.strip() for p in (raw or '').replace(' - ',',').split(',') if p.strip()]
    if parts and _find_city(parts[0]):
        mun=parts[0].title()
    return {
        'region':region,'municipality':mun,'zone':zone,
        'location_details':{'region':region,'municipality':mun,'zone':zone,'raw':raw}
    }

def get_conn():
    return psycopg2.connect(**DB_CONFIG)

def url_exists(cur,u):
    cur.execute('SELECT 1 FROM fi_dari_listings WHERE url=%s',(u,))
    return cur.fetchone() is not None

def extract_rooms(t):
    if not t:
        return None
    t=t.lower()
    for pat in [r'\bs\s*\+?\s*(\d)\b',r'\b[ft]\s*(\d)\b',r'(\d)\s*pi[ee]ces?',r'\bstudio\b']:
        m=re.search(pat,t)
        if m:
            if 'studio' in pat:
                return 1
            v=int(m.group(1))
            return v if 1<=v<=15 else None
    return None

def extract_surface(t):
    if not t:
        return None
    m=re.search(r'([\d][\d\s.,]*)\s*m\s*[2]',t.lower())
    if m:
        try:
            v=float(m.group(1).replace(' ','').replace(',','.'))
            return v if 10<=v<=50000 else None
        except:
            pass
    return None

def extract_price(t):
    if not t:
        return None
    t=re.sub(r'(?i)\b(tnd|dt|dinar)\b','',str(t))
    t=t.replace('\xa0','').replace('\u202f','')
    d=re.sub(r'[^\d.,]','',t.strip())
    if not d:
        return None
    if re.match(r'^\d{1,3}[.,]\d{3}$',d):
        d=d.replace('.','').replace(',','')
    elif d.count('.')>1:
        d=d.replace('.','')
    d=d.replace(',','.')
    try:
        v=float(d)
        return v if 500<=v<=100_000_000 else None
    except:
        return None

def download_images(urls,ptype,lid):
    folder=os.path.join(MEDIA_ROOT,ptype,str(lid))
    os.makedirs(folder,exist_ok=True)
    if not urls:
        return None,folder,0
    saved=[]
    hdrs={'User-Agent':'Mozilla/5.0'}
    try:
        from PIL import Image as _PI
        import io as _io
        _pil=True
    except:
        _pil=False
    for url in dict.fromkeys(u for u in urls if u and not u.startswith('data:')):
        if _is_parasite(url) or not _is_photo(url):
            continue
        uhd=re.sub(r'[_-](thumb|small|medium)','',url,flags=re.I)
        for u in [uhd,url]:
            try:
                r=requests.get(u,headers=hdrs,timeout=12)
                if r.status_code==200 and len(r.content)>=40_000:
                    if _pil:
                        try:
                            img=_PI.open(_io.BytesIO(r.content))
                            if img.size[0]<300 or img.size[1]<200:
                                break
                        except:
                            pass
                    ct=r.headers.get('Content-Type','')
                    ext='webp' if 'webp' in ct else 'png' if 'png' in ct else 'jpg'
                    fp=os.path.join(folder,f'image_{len(saved)+1:03d}.{ext}')
                    with open(fp,'wb') as f:
                        f.write(r.content)
                    saved.append(fp)
                    time.sleep(0.2)
                    break
            except:
                continue
    return (saved[0] if saved else None),folder,len(saved)

def insert_listing(cur,d):
    try:
        cur.execute(
            'INSERT INTO fi_dari_listings(title,price,transaction_type,type,region,municipality,zone,location_details,surface,rooms,features,poi,description,url,pdf_link,image_path,images_folder,images_count,last_updated,is_new) VALUES(%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s) ON CONFLICT(url) DO NOTHING RETURNING id',
            (d.get('title'),d.get('price'),d.get('transaction_type'),d.get('type'),
             d.get('region'),d.get('municipality'),d.get('zone'),json.dumps(d.get('location_details',{})),
             d.get('surface'),d.get('rooms'),json.dumps(d.get('features',{})),json.dumps(d.get('poi',{})),
             d.get('description',''),d.get('url'),d.get('pdf_link'),None,None,0,d.get('last_updated','Unknown'),True))
        row=cur.fetchone()
        return row[0] if row else None
    except Exception as e:
        print(f'Insert err:{e}')
        return None

def update_images(cur,lid,t,f,c):
    cur.execute('UPDATE fi_dari_listings SET image_path=%s,images_folder=%s,images_count=%s WHERE id=%s',(t,f,c,lid))

def log_start(cur,rt):
    cur.execute('INSERT INTO fi_dari_log(run_type) VALUES(%s) RETURNING id',(rt,))
    return cur.fetchone()[0]

def log_end(cur,lid,n,e,s='done'):
    cur.execute('UPDATE fi_dari_log SET finished_at=NOW(),total_new=%s,total_errors=%s,status=%s WHERE id=%s',(n,e,s,lid))

def reset_is_new(cur):
    cur.execute('UPDATE fi_dari_listings SET is_new=FALSE')

def create_driver():
    opts=Options()
    opts.add_argument('--headless=new')
    opts.add_argument('--no-sandbox')
    opts.add_argument('--disable-dev-shm-usage')
    opts.add_argument('--window-size=1920,1080')
    opts.add_experimental_option('excludeSwitches',['enable-automation'])
    opts.add_argument('user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 Chrome/120.0.0.0 Safari/537.36')
    opts.set_capability('goog:loggingPrefs',{'performance':'ALL'})
    d=webdriver.Chrome(service=Service(ChromeDriverManager().install()),options=opts)
    d.execute_script("Object.defineProperty(navigator,'webdriver',{get:()=>undefined})")
    return d

def deep_scroll(driver,wait=10):
    time.sleep(wait)
    total=driver.execute_script('return document.body.scrollHeight')
    step=max(200,total//15)
    pos=0
    while pos<total:
        driver.execute_script(f'window.scrollTo(0,{pos});')
        time.sleep(0.3)
        pos+=step
        nh=driver.execute_script('return document.body.scrollHeight')
        if nh>total:
            total=nh
    driver.execute_script('window.scrollTo(0,document.body.scrollHeight);')
    time.sleep(2)
    driver.execute_script('window.scrollTo(0,0);')
    time.sleep(1)

def get_annonces(driver,url):
    driver.get(url)
    deep_scroll(driver,10)
    soup=BeautifulSoup(driver.page_source,'html.parser')
    urls=set()
    for a in soup.find_all('a',href=True):
        h=a['href']
        if any(p in h for p in ['/bien/','/annonce/','/detail/']):
            if not h.startswith('http'):
                h=BASE_URL+h
            urls.add(h)
    return [{'url':u,'title_hint':'','price_hint':'','location_hint':'','thumb':None} for u in urls]

def scrape_detail(driver,url):
    try:
        driver.get(url)
        deep_scroll(driver,7)
        soup=BeautifulSoup(driver.page_source,'html.parser')
        data={}
        h1=soup.find('h1')
        data['title_detail']=h1.get_text(strip=True) if h1 else ''
        desc=''; best=0
        for tag in soup.find_all(['div','p','section']):
            cs=' '.join(tag.get('class',[]))
            if any(x in cs.lower() for x in ['description','detail','content','text','body']):
                t=tag.get_text(' ',strip=True)
                if 30<len(t)<5000 and len(t)>best:
                    best=len(t); desc=t
        data['description']=desc
        ptext=re.compile(r'([\d][\d\s.,]*)[\s]*(?:dt|tnd|dinar)',re.I)
        m=ptext.search(soup.get_text(' '))
        if m:
            data['price']=extract_price(m.group(1))
        for tag in soup.find_all('time'):
            data['last_updated']=tag.get('datetime','') or tag.get_text(strip=True)
            break
        data.setdefault('last_updated','Unknown')
        for a in soup.find_all('a',href=True):
            h=a['href']
            txt=a.get_text(strip=True).lower()
            if h.lower().endswith('.pdf') or any(k in txt for k in ['fiche','telecharger','pdf']):
                data['pdf_link']=h if h.startswith('http') else BASE_URL+'/'+h.lstrip('/')
                break
        data.setdefault('pdf_link',None)
        loc=''
        for nav in soup.find_all(['nav','ol'],class_=re.compile(r'breadcrumb|ariane',re.I)):
            parts=[i.get_text(strip=True) for i in nav.find_all(['li','a','span']) if i.get_text(strip=True)]
            geo=[p for p in parts[1:-1] if 2<len(p)<60]
            if geo:
                loc=' - '.join(geo)
                break
        if not loc:
            m2=re.search(r'/bien/([^/?#]+)',url)
            if m2:
                loc=m2.group(1).replace('-',' ')
        data['location_raw']=loc
        imgs=[]; seen=set()
        def add(src):
            if not src or len(src)<15:
                return
            if src.startswith('//'):
                src='https:'+src
            if not src.startswith('http'):
                src=BASE_URL+'/'+src.lstrip('/')
            if not _is_parasite(src) and _is_photo(src) and src not in seen:
                seen.add(src); imgs.append(src)
        for img in soup.find_all('img'):
            for attr in ['data-big','data-original','data-src','src']:
                src=img.get(attr,'')
                if src:
                    add(src)
                    break
        data['img_urls']=imgs
        data['surface_detail']=extract_surface(desc)
        data['rooms_detail']=extract_rooms(desc)
        result={}
        for li in soup.find_all('li'):
            t=li.get_text(strip=True)
            if 2<len(t)<80:
                for fr,en in FEATURE_MAPPING.items():
                    if fr.lower() in t.lower():
                        result[en]=True
                        break
        data['features']=result
        return data
    except Exception as e:
        print(f'detail err:{e}')
        return {}

def build_url(slug,page):
    base=f'{BASE_URL}/{slug}'
    base=re.sub(r'&page=\d+','',base)
    return f'{base}&page={page}'

def run_daily():
    now=datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    sep='='*60
    print(f'\n{sep}\n  CYCLE QUOTIDIEN Fi-Dari - {now}\n{sep}')
    os.makedirs(MEDIA_ROOT,exist_ok=True)
    conn=get_conn(); cur=conn.cursor(); driver=create_driver()
    log_id=log_start(cur,'daily'); reset_is_new(cur); conn.commit()
    total_new=0; total_errors=0
    try:
        for slug,(tt,pt) in CATEGORIES.items():
            cat_new=0
            print(f'\n  {pt} | {tt}')
            for page_num in range(1,MAX_PAGES_DAILY+1):
                url_page=build_url(slug,page_num)
                anns=[]
                try:
                    anns=get_annonces(driver,url_page)
                except:
                    driver=create_driver()
                if not anns:
                    break
                new_this=0
                for ann in anns:
                    ann_url=ann.get('url','')
                    if not ann_url or url_exists(cur,ann_url):
                        continue
                    print(f'    NEW {ann_url[-60:]}')
                    det={}
                    try:
                        det=scrape_detail(driver,ann_url)
                    except:
                        pass
                    price=det.get('price') or extract_price(ann.get('price_hint',''))
                    loc_raw=det.get('location_raw','') or ann.get('location_hint','')
                    desc=det.get('description','')
                    title=det.get('title_detail','') or 'N/A'
                    geo=build_location(loc_raw,title,desc)
                    cfg=PROPERTY_CONFIG.get(pt,{'has_rooms':True,'has_surface':True})
                    surface=(det.get('surface_detail') or extract_surface(desc)) if cfg['has_surface'] else None
                    rooms=(det.get('rooms_detail') or extract_rooms(desc)) if cfg['has_rooms'] else None
                    data={
                        'title':title,'price':price,'transaction_type':tt,'type':pt,
                        **{k:geo[k] for k in ['region','municipality','zone','location_details']},
                        'surface':surface,'rooms':rooms,'features':det.get('features',{}),'poi':{},
                        'description':desc,'url':ann_url,'pdf_link':det.get('pdf_link'),
                        'last_updated':det.get('last_updated','Unknown')
                    }
                    lid=insert_listing(cur,data)
                    if not lid:
                        total_errors+=1
                        continue
                    conn.commit()
                    imgs=list(det.get('img_urls',[]))
                    if ann.get('thumb') and not _is_parasite(ann['thumb']) and _is_photo(ann['thumb']):
                        if ann['thumb'] not in imgs:
                            imgs.insert(0,ann['thumb'])
                    t,f,n=download_images(imgs,pt,lid)
                    update_images(cur,lid,t,f,n)
                    conn.commit()
                    cat_new+=1; total_new+=1; new_this+=1
                    mun=geo['municipality']
                    rgn=geo['region']
                    print(f'    OK id={lid} {mun},{rgn} prix={price} imgs={n}')
                    time.sleep(random.uniform(2,4))
                if new_this==0:
                    break
                time.sleep(random.uniform(2,5))
            print(f'  {pt}_{tt} -> {cat_new} nouvelles')
    except Exception as e:
        print(f'ERREUR:{e}')
        log_end(cur,log_id,total_new,total_errors,'error')
        conn.commit()
    else:
        log_end(cur,log_id,total_new,total_errors,'done')
        conn.commit()
    finally:
        try:
            driver.quit()
        except:
            pass
        cur.close()
        conn.close()
    print(f'\n  TERMINE - {total_new} nouvelles annonces')

if __name__=='__main__':
    run_daily()
"""

with open(DAILY_PY, 'w', encoding='utf-8') as f:
    f.write(DAILY_CONTENT)
print(f"OK {DAILY_PY}")

bat_content = (
    "@echo off\n"
    "echo ==========================================\n"
    f"echo  Fi-Dari Daily Scraper - %date% %time%\n"
    "echo ==========================================\n"
    f'cd /d "{PROJECT_DIR}"\n'
    f'"{PYTHON_EXE}" daily_scrape.py >> "logs\\daily_log.txt" 2>&1\n'
    f'echo Termine : %date% %time% >> "logs\\daily_log.txt"\n'
)
with open(BAT_FILE, 'w', encoding='utf-8') as f:
    f.write(bat_content)
print(f"OK {BAT_FILE}")

OK B:\EstateMind\fi_dari_scraper\daily_scrape.py
OK B:\EstateMind\fi_dari_scraper\lancer_daily.bat


In [11]:
# ═══════════════════════════════════════════════════════════
# CELLULE 9 — STATISTIQUES
# ═══════════════════════════════════════════════════════════
import warnings
warnings.filterwarnings('ignore')
import pandas as pd

conn = psycopg2.connect(**DB_CONFIG)

print("═"*65); print("  📊 STATISTIQUES PAR CATÉGORIE"); print("═"*65)
pd.read_sql("""
    SELECT type, transaction_type,
           COUNT(*)                       AS total,
           COUNT(price)                   AS avec_prix,
           ROUND(AVG(price)::numeric, 0)  AS prix_moyen,
           COUNT(rooms)                   AS avec_rooms,
           COUNT(surface)                 AS avec_surface,
           ROUND(AVG(surface)::numeric,1) AS surface_moy,
           COUNT(pdf_link)                AS avec_pdf,
           SUM(images_count)              AS total_images
    FROM fi_dari_listings
    GROUP BY type, transaction_type
    ORDER BY type, transaction_type
""", conn).pipe(display)

print("\n"+"═"*65); print("  🆕 20 DERNIÈRES ANNONCES"); print("═"*65)
pd.read_sql("""
    SELECT id, title, type, transaction_type,
           rooms, surface, price,
           region, municipality, zone,
           images_count, last_updated, scraped_at
    FROM fi_dari_listings
    ORDER BY scraped_at DESC LIMIT 20
""", conn).pipe(display)

print("\n"+"═"*65); print("  🗺️  RÉPARTITION GÉOGRAPHIQUE"); print("═"*65)
pd.read_sql("""
    SELECT zone, region, municipality,
           COUNT(*) AS total
    FROM fi_dari_listings
    GROUP BY zone, region, municipality
    ORDER BY total DESC LIMIT 20
""", conn).pipe(display)

print("\n"+"═"*65); print("  🖼️  TOP ANNONCES PAR IMAGES"); print("═"*65)
pd.read_sql("""
    SELECT id, type, title, images_count, images_folder
    FROM fi_dari_listings
    WHERE images_count > 0
    ORDER BY images_count DESC LIMIT 10
""", conn).pipe(display)

print("\n"+"═"*65); print("  📋 HISTORIQUE CYCLES"); print("═"*65)
pd.read_sql("""
    SELECT run_type, started_at, finished_at,
           total_new, total_errors, status
    FROM fi_dari_log
    ORDER BY started_at DESC LIMIT 10
""", conn).pipe(display)

conn.close()


═════════════════════════════════════════════════════════════════
  📊 STATISTIQUES PAR CATÉGORIE
═════════════════════════════════════════════════════════════════


,type,transaction_type,total,avec_prix,prix_moyen,avec_rooms,avec_surface,surface_moy,avec_pdf,total_images
0,apartment,rent,200,198,2282.0,200,101,101.4,0,2925
1,apartment,sale,200,190,328031.0,200,200,455.3,0,4166
2,house,rent,148,120,2730.0,128,108,170.9,0,1459
3,house,sale,200,199,333258.0,200,200,193.0,0,4046
4,land,rent,5,4,279550.0,0,4,2520.0,0,52
5,land,sale,201,199,626736.0,0,198,1064.3,2,3004
6,office,rent,200,194,117218.0,0,192,469.5,2,1744
7,office,sale,24,17,2700691.0,0,24,872.0,0,328



═════════════════════════════════════════════════════════════════
  🆕 20 DERNIÈRES ANNONCES
═════════════════════════════════════════════════════════════════


,id,title,type,transaction_type,rooms,surface,price,region,municipality,zone,images_count,last_updated,scraped_at
0,1178,Achat terrain de 145m²,land,sale,None,145.0,170000.0,Bizerte,Bizerte Pa32070,nord,0,Unknown,2026-02-21 22:50:28.040989
1,1177,Location bureau de 135m²,office,rent,None,135.0,NaN,Ariana,Soukra,grand_tunis,9,9 septembre 2024,2026-02-21 21:17:13.662477
2,1176,Location bureau de 250m²,office,rent,None,250.0,8000.0,Tunis,Tunis,grand_tunis,8,11 septembre 2024,2026-02-21 21:16:50.185808
3,1175,Location bureau de 114m²,office,rent,None,114.0,2300.0,Tunis,Tunis,grand_tunis,4,12 septembre 2024,2026-02-21 21:16:29.608177
4,1174,Location bureau de 101m²,office,rent,None,101.0,2100.0,Tunis,Tunis,grand_tunis,4,12 septembre 2024,2026-02-21 21:16:08.094144
5,1173,Location bureau de 107m²,office,rent,None,107.0,2200.0,Tunis,Centre Urbain Nord,grand_tunis,5,10 septembre 2024,2026-02-21 21:15:46.879898
6,1172,Location bureau de 70m²,office,rent,None,70.0,1900.0,Tunis,Tunis,grand_tunis,6,4 septembre 2024,2026-02-21 21:15:25.080956
7,1171,Location bureau de 75m²,office,rent,None,75.0,1700.0,Tunis,Tunis,grand_tunis,4,11 octobre 2024,2026-02-21 21:15:04.391276
8,1170,Location bureau de 350m²,office,rent,None,350.0,8500.0,Tunis,Tunis,grand_tunis,4,10 septembre 2024,2026-02-21 21:14:44.410577
9,1169,Location bureau de 65m²,office,rent,None,65.0,1950.0,Tunis,Tunis,grand_tunis,3,13 septembre 2024,2026-02-21 21:14:23.979872



═════════════════════════════════════════════════════════════════
  🗺️  RÉPARTITION GÉOGRAPHIQUE
═════════════════════════════════════════════════════════════════


,zone,region,municipality,total
0,nord,Nabeul,Nabeul,333
1,grand_tunis,Tunis,Tunis,170
2,grand_tunis,Tunis,La Marsa,84
3,nord,Nabeul,Hammamet,65
4,grand_tunis,Ariana,Soukra,50
5,nord,Nabeul,Beni Khiar,49
6,grand_tunis,Tunis,Carthage,49
7,nord,Nabeul,Hammamet Nord,43
8,grand_tunis,Ariana,El Menzah,37
9,grand_tunis,Ariana,Mrezga,30



═════════════════════════════════════════════════════════════════
  🖼️  TOP ANNONCES PAR IMAGES
═════════════════════════════════════════════════════════════════


,id,type,title,images_count,images_folder
0,587,house,Achat maison S+6 de 280m²,48,C:\Users\chtou\Desktop\scrapping\media_fidari\...
1,386,apartment,Location appartement s+2 de 80m²,44,C:\Users\chtou\Desktop\scrapping\media_fidari\...
2,648,house,Location maison s+4 de 200m²,44,C:\Users\chtou\Desktop\scrapping\media_fidari\...
3,610,house,Location maison s+3 de 100m²,42,C:\Users\chtou\Desktop\scrapping\media_fidari\...
4,604,house,Location maison s+3 de 370m²,42,C:\Users\chtou\Desktop\scrapping\media_fidari\...
5,1042,office,Location bureau de 250m²,41,C:\Users\chtou\Desktop\scrapping\media_fidari\...
6,81,apartment,Achat appartement s+3 de 159m²,41,C:\Users\chtou\Desktop\scrapping\media_fidari\...
7,227,apartment,Location appartement s+3 de 200m²,40,C:\Users\chtou\Desktop\scrapping\media_fidari\...
8,435,house,Achat maison s+2 de 180m²,39,C:\Users\chtou\Desktop\scrapping\media_fidari\...
9,217,apartment,Location appartement s+3 de 130m²,38,C:\Users\chtou\Desktop\scrapping\media_fidari\...



═════════════════════════════════════════════════════════════════
  📋 HISTORIQUE CYCLES
═════════════════════════════════════════════════════════════════


,run_type,started_at,finished_at,total_new,total_errors,status
0,daily,2026-02-21 23:06:42.891916,2026-02-21 23:07:02.910761,0,0,done
1,daily,2026-02-21 22:54:29.812294,2026-02-21 22:54:49.975220,0,0,done
2,daily,2026-02-21 22:49:47.348499,2026-02-21 22:52:14.797525,1,0,done
3,initial,2026-02-21 11:41:13.644112,2026-02-21 21:17:40.656249,1177,0,done
